**批量提取某标签下所有子孙节点标签的NavigableString**

对于单个节点的内容可以利用 .string 或 .text 来获得，若想获得多个节点内容：

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(open('html.html','r',encoding='utf-8'), 'lxml')
soup

In [ ]:
soup.a.parent.parent

In [ ]:
soup.ul.text

In [ ]:
for t in soup.ul.strings:
    print(repr(t))    # repr() 函数将对象转化为供解释器读取的形式

In [ ]:
for t in soup.ul.stripped_strings:
    print(t)

#### 信息提取的方法

一：完整解析信息的标记形式，再提取关键信息
- 需要标记解析器，例如：bs4库的标签树遍历
- 优点：信息解析准确
- 缺点：提取过程繁琐，速度慢

二：无视标记形式，直接搜索关键信息
- 对信息的文本查找函数即可
- 优点：提取过程简洁，速度较快
- 缺点：提取结果准确性与信息内容相关

#### 基于bs4库的html内容遍历方法

![](spider02.jpg)



**上行遍历**
+ .parent：返回当前节点的父节点标签
+ .parents：返回当前节点所有先辈节点标签的迭代类型，仅用于循环遍历

In [ ]:
soup.a.parent

In [ ]:
for adult in soup.a.parents:
    print(adult.name)

**下行遍历**
+ .contents：返回子节点标签的列表
+ .children：返回子节点标签的迭代类型，仅用于循环遍历
+ .descendants：返回包含所有子孙节点标签的迭代类型，仅用于循环遍历

In [ ]:
type(soup.body.contents)
soup.body.contents

In [ ]:
type(soup.body.children)
for child in soup.body.children:
    print(child)

In [ ]:
type(soup.body.descendants)

**平行遍历（按照HTML文本顺序）**
+ .next_sibling：返回下一个平行节点标签
+ .previous_sibling：返回上一个平行节点标签
+ .next_siblings：返回后续所有平行节点标签的迭代类型
+ .previous_siblings：返回前序所有平行节点标签的迭代类型

Notice：实际html标签树的平行标签通常是空白/回车/tab等，因此通常仅使用.next_siblings 和 .previous_siblings循环输出当前节点的兄弟节点。

In [ ]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://www.baidu.com/')
res.encoding='utf-8'
soup = BeautifulSoup(res.text,'lxml')

for sibling in soup.a.next_siblings:
    print(repr(sibling))

**Exercise：**

请大家提取出百度主页的html页面的全部标签名称。返回的结果为列表，不包含None并去除重复元素。

In [ ]:
#### Answer
import requests
from bs4 import BeautifulSoup

res = requests.get('https://www.baidu.com/')
res.encoding='utf-8'
soup = BeautifulSoup(res.text,'lxml')
tagname=[i.name for i in soup.html.descendants]
tagname=list(set(filter(None,tagname)))
tagname

#### 正则表达式(regular expression, regex, RE)

>正则表达式是一种通用的字符串表达框架，可以用来简洁地表达一组字符串，也可以用来判断某字符串的特征归属。

正则表达式在文本处理中十分常用：
- 表达文本类型的特征
- 查找或替换一组字符串
- 匹配字符串的全部或部分

**正则表达式的常用操作符**
```
.    匹配除换行符以外的任意字符
[  ] 字符集，对单个字符给出取值范围
[^ ] 非字符集，对单个字符给出排除范围
^    匹配字符串开头
$    匹配字符串结尾
\d   匹配数字，等价于[0‐9]
\w   匹配字母或数字或下划线，等价于[A‐Za‐z0‐9_]

```

**re库的使用**

>Re库是Python的标准库，主要用于字符串匹配。调用方式：import re

`regex = re.compile(pattern, flags=0)`

`将正则表达式的字符串形式编译成正则表达式对象`
```
pattern : 正则表达式的字符串或原生字符串表示
flags : 正则表达式使用时的控制标记
```



#### 基于bs4库的html内容查找方法

`<>.find_all(name, attrs, recursive, string, **kwargs)`  [官方文档](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/#find-all)

`返回一个列表类型，存储查找的结果`
```
name: 查找所有名字为 name 的标签，name 参数的值可以是字符串，正则表达式，列表或是True。
attrs: 对标签属性值的进行检索
recursive: 是否对子孙节点全部检索，默认True，若设为False则仅检索子节点
string: 对<>…</>中字符串区域的检索
```

**检索标签名称**

- 传入字符串：查找与字符串完整匹配的内容

In [ ]:
soup.find_all('a')

- 传入列表：返回与列表中任一元素匹配的内容

In [ ]:
soup.find_all(['a','p'])

- 传入True：匹配任何值

In [ ]:
for tag in soup.find_all(True):
    print(tag.name)

- 传入正则表达式

In [ ]:
import re
for tag in soup.find_all(re.compile('t')):  #### 返回所有包含t的标签名
    print(tag.name)

**检索标签属性**

In [ ]:
soup.find_all(attrs={'target':'_blank'})

In [ ]:
soup.find_all(target='_blank')

In [ ]:
soup.find_all(href=re.compile("page"))

In [ ]:
soup.find_all(href=True)

**string参数**

通过 string 参数可以搜索文档中的字符串内容。string 参数接受字符串、正则表达式、列表、True。

In [ ]:
soup.find_all(string=[re.compile("利好"), re.compile("新高"), re.compile("扩大")])

In [ ]:
soup.find_all('a',string=[re.compile("利好"), re.compile("新高"), re.compile("扩大")])

**limit参数**

find_all()方法返回全部的搜索结构，如果文档树很大那么搜索会很慢。如果我们不需要全部结果，可以使用 limit 参数限制返回结果的数量。

In [ ]:
soup.find_all(string=[re.compile("利好"), re.compile("新高"), re.compile("扩大")], limit = 3)

**find_all的扩展方法**

- <>.find()：搜索且只返回一个结果
- <>.find_parents()：在先辈节点中搜索，返回列表类型
- <>.find_parent()：在先辈节点中返回一个结果
- <>.find_next_siblings()：在后续平行节点中搜索，返回列表类型
- <>.find_next_sibling()：在后续平行节点中返回一个结果
- <>.find_previous_siblings()：在前序平行节点中搜索，返回列表类型
- <>.find_previous_sibling()：在前序平行节点中返回一个结果

区别：检索区域和返回结果不同

具体请参阅文档 https://beautifulsoup.readthedocs.io/zh_CN/latest/

**Exercise：**

1. 请提取页面中的股票代票、股票名称、公告日期这三项信息，并将数据存放在dataframe中。
2. 请提取出下述url页面中红色的数值。

In [1]:
import requests
from bs4 import BeautifulSoup
import re

## 使用一个网易的数据页面
url="http://quotes.money.163.com/data/caibao/yjgl_ALL.html?reportdate=20200930&sort=publishdate&order=desc&page=0"
     
def request_url(url):
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.109 Safari/537.36'
    headers = {'User-Agent': user_agent} 
    
    res = requests.get(url,headers=headers)
    res.encoding = 'utf-8'
    return res.text

#### 提取三项信息并将数据存入dataframe中

soup = BeautifulSoup(request_url(url), 'html.parser')
# len(soup.find_all(table))
soup.table.find_all('a')
text = [i.text for i in soup.table.find_all('a', href = re.compile('#11a01'))]
code = text[::2]
name = text[1::2]
date = [i.strip() for i in soup.table.find_all(string=re.compile('2020'))]

import pandas as pd

data = pd.DataFrame({'代码':code, '名称':name, '日期': date})
data

,代码,名称,日期
0,300923,研奥股份,2020-12-09
1,605377,华旺科技,2020-12-09
2,605179,一鸣食品,2020-12-08
3,003028,振邦智能,2020-12-07
4,003029,吉大正元,2020-12-07
5,688658,悦康药业,2020-12-04
6,688678,福立旺,2020-12-03
7,688698,伟创电气,2020-12-02
8,605186,健麾信息,2020-12-01
9,003026,中晶科技,2020-12-01


In [7]:
#### 提取页面中红色的数值
[x.text for x in soup.find_all('td', class_='red')]

['-6.70%',
 '-12.71%',
 '-5.67%',
 '-20.05%',
 '-2.75%',
 '-6.39%',
 '-0.03',
 '-2,495.65万',
 '-2.20%',
 '-0.47',
 '-77.70%',
 '-14.80亿',
 '-11.03%',
 '-0.45%',
 '-9.27%',
 '-5.83%']

3. 基于上述实例，尝试通过修改url = 'https://finance.sina.com.cn/roll/index.d.html?cid=56588&page=1'
中的关键词 page，实现自动翻页爬取多条信息，并存入本地数据库中。



**Exercise：**

1. 下面是一个批量下载百度图片的例子，请通过修改num，下载更多的图片，将num作为循环变量的参数写入程序中。
2. 当下载的图片很多时会发生各种错误，如何让程序稳定地运行？


In [ ]:
import requests
import re
import os
import time

pathname = 'D:\\图片\\'
if not os.path.exists(pathname):
    os.makedirs(pathname)
name = input('您要爬取的图片名称：')
num = 1
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'}
url = 'https://image.baidu.com/search/flip?tn=baiduimage&ie=utf-8&word='+name+'&pn='+str((num-1)*60)
res = requests.get(url,headers=headers)
# print(res.request.url)

html = res.content.decode()
addlist = re.findall('"objURL":"(.*?)",',html)
# print(len(addlist))
# print(addlist)

j=0
for i in addlist:
    j = j +1
    img = requests.get(i)
    f = open(pathname+name+str(j)+'.jpg','ab')
    print('---------正在下载第'+str(j)+'张图片----------')
    f.write(img.content)
    f.close()
print('下载完成')

**延申：**

1. 在之前的课程中，我们提到过另一种解析器lxml，它提供另一种方法XPath来提取网页中的数据，感兴趣的同学可参阅[click](https://www.cnblogs.com/cathycheng/p/11422685.html)

2. 思考如何爬取动态网页。

#### 小结：

1. 提取NavigableString的几种方法：text、strings、stripped_strings

2. 遍历网页标签树：上行，下行，平行

3. 利用find_all方法搜索标签树：对标签名称、属性、内容的搜索（正则表达式）
